sub-BH243.nwb
2 days ago1.09 MB
sub-BH243_ses-20211103_obj-1dppqzr.nwb
2 days ago1.09 MB
sub-BH243_ses-20211103_obj-1okdjl.nwb
2 days ago1.04 MB
sub-BH243_ses-20211103_obj-hmtn3m.nwb
2 days ago1.04 MB
sub-BH243_ses-20211103_obj-zp74wq.nwb

In [13]:
from pynwb import NWBHDF5IO

# Define the file path
input_file_path = 'data\sub-BH243_ses-20211103_obj-1okdjl.nwb'

# Read the NWB file
with NWBHDF5IO(input_file_path, 'r') as io:
    nwbfile = io.read()
    
    # List the top-level data groups
    print("Available processing modules:", nwbfile.processing.keys())
    
    # If "ecephys" is not available, print other data interfaces
    for module_name, module in nwbfile.processing.items():
        print(f"Module: {module_name}")
        for name, data_interface in module.data_interfaces.items():
            print(f"  Data Interface: {name} - {type(data_interface)}")


Available processing modules: dict_keys([])


In [14]:
from pynwb import NWBHDF5IO

# Define the file path
input_file_path = 'data\sub-BH243_ses-20211103_obj-1okdjl.nwb'

# Read the NWB file
with NWBHDF5IO(input_file_path, 'r') as io:
    nwbfile = io.read()

    # Explore the acquisition module
    print("Available acquisition data:")
    for name, data_interface in nwbfile.acquisition.items():
        print(f"  Name: {name}, Type: {type(data_interface)}")
    
    # Explore other potential sections
    print("\nOther available data interfaces:")
    print("  Analysis:", nwbfile.analysis)
    print("  Stimulus:", nwbfile.stimulus)
    print("  Scratch:", nwbfile.scratch)

Available acquisition data:

Other available data interfaces:
  Analysis: {}
  Stimulus: {}
  Scratch: {}


In [15]:
import h5py

# Define the file path
input_file_path = 'data\sub-BH243_ses-20211103_obj-1okdjl.nwb'

# Open the NWB file as an HDF5 file and recursively list its contents
def explore_hdf5_group(group, path='/'):
    for key in group.keys():
        item = group[key]
        if isinstance(item, h5py.Group):
            print(f"Group: {path}{key}")
            explore_hdf5_group(item, path=path + key + '/')
        elif isinstance(item, h5py.Dataset):
            print(f"Dataset: {path}{key} - Shape: {item.shape}, Dtype: {item.dtype}")

with h5py.File(input_file_path, 'r') as f:
    explore_hdf5_group(f)


Group: /acquisition
Group: /analysis
Dataset: /file_create_date - Shape: (1,), Dtype: object
Group: /general
Group: /general/devices
Group: /general/extracellular_ephys
Dataset: /general/institution - Shape: (), Dtype: object
Group: /general/intracellular_ephys
Group: /general/optogenetics
Group: /general/optophysiology
Group: /general/subject
Dataset: /general/subject/age - Shape: (), Dtype: object
Dataset: /general/subject/description - Shape: (), Dtype: object
Dataset: /general/subject/sex - Shape: (), Dtype: object
Dataset: /general/subject/species - Shape: (), Dtype: object
Dataset: /general/subject/subject_id - Shape: (), Dtype: object
Dataset: /identifier - Shape: (), Dtype: object
Group: /intervals
Group: /processing
Group: /scratch
Dataset: /session_description - Shape: (), Dtype: object
Dataset: /session_start_time - Shape: (), Dtype: object
Group: /specifications
Group: /specifications/core
Group: /specifications/core/2.4.0
Dataset: /specifications/core/2.4.0/namespace - Sha

In [16]:
from pynwb import NWBHDF5IO
import pyedflib
import numpy as np

# Define the file paths
input_file_path = 'data\sub-BH243_ses-20211103_obj-1okdjl.nwb'  # Update with your NWB file path
output_file_path = 'data\sub-BH243_ses-20211103_obj-1okdjl.edf'

# Read the NWB file
with NWBHDF5IO(input_file_path, 'r') as io:
    nwbfile = io.read()
    
    # Extract data and metadata (example assumes a single time series)
    if 'ecephys' in nwbfile.processing:
        ecephys = nwbfile.processing['ecephys']
        data_interface = next(iter(ecephys.data_interfaces.values()))
        data = data_interface.data[:]
        timestamps = data_interface.timestamps[:]
        channel_names = data_interface.electrodes['label'][:]
    else:
        raise ValueError("NWB file does not contain ecephys data.")

    # Prepare EDF file writer
    n_channels = len(channel_names)
    signal_headers = []
    signal_data = []

    for i, channel in enumerate(channel_names):
        signal_headers.append({
            'label': channel,
            'dimension': 'uV',
            'sample_rate': 1 / np.mean(np.diff(timestamps)),
            'physical_max': np.max(data[:, i]),
            'physical_min': np.min(data[:, i]),
            'digital_max': 32767,
            'digital_min': -32768,
            'transducer': '',
            'prefilter': ''
        })
        # Scale data to fit EDF limits
        scaled_data = np.int16((data[:, i] - np.min(data[:, i])) /
                               (np.max(data[:, i]) - np.min(data[:, i])) * (32767 - (-32768)) + (-32768))
        signal_data.append(scaled_data)

    # Write to EDF file
    with pyedflib.EdfWriter(output_file_path, n_channels=n_channels, file_type=pyedflib.FILETYPE_EDFPLUS) as edf:
        edf.setSignalHeaders(signal_headers)
        edf.writeSamples(signal_data)

print("Conversion to EDF format completed successfully.")


ValueError: NWB file does not contain ecephys data.

In [17]:
import numpy as np
import pyedflib

# Define parameters for binary signal representation
sampling_frequency = 1000  # 1 kHz sampling frequency
output_file_path = 'sub-BH243_spike_binary.edf'

# Example spike data (replace with real data)
spike_times = np.random.uniform(0, 10, 117397)  # Simulated spike times (in seconds)
spike_times_index = np.arange(20) * 5870  # Simulated spike indices for each unit
num_units = len(spike_times_index)  # Number of units
total_time = spike_times.max()  # Total duration of the recording in seconds

# Create a time axis for the binary signals
num_samples = int(total_time * sampling_frequency)
binary_signals = np.zeros((num_units, num_samples), dtype=np.int16)

# Populate binary signals with spikes
for unit_idx in range(num_units):
    start_idx = spike_times_index[unit_idx]
    end_idx = spike_times_index[unit_idx + 1] if unit_idx + 1 < num_units else len(spike_times)
    unit_spikes = spike_times[start_idx:end_idx]
    spike_indices = (unit_spikes * sampling_frequency).astype(int)
    spike_indices = np.clip(spike_indices, 0, num_samples - 1)  # Clip indices to valid range
    binary_signals[unit_idx, spike_indices] = 1

# Create EDF signal headers
signal_headers = []
for unit_idx in range(num_units):
    signal_headers.append({
        'label': f'Unit {unit_idx}',
        'dimension': '',
        'sample_rate': sampling_frequency,
        'physical_max': 1,
        'physical_min': 0,
        'digital_max': 1,
        'digital_min': 0,
        'transducer': '',
        'prefilter': ''
    })

# Write binary spike data to EDF file
with pyedflib.EdfWriter(output_file_path, n_channels=num_units, file_type=pyedflib.FILETYPE_EDFPLUS) as edf:
    edf.setSignalHeaders(signal_headers)
    edf.writeSamples(binary_signals.tolist())

print("EDF file with binary spike data created successfully.")


AttributeError: 'list' object has no attribute 'flags'

In [18]:
# Write binary spike data to EDF file
with pyedflib.EdfWriter(output_file_path, n_channels=num_units, file_type=pyedflib.FILETYPE_EDFPLUS) as edf:
    edf.setSignalHeaders(signal_headers)
    edf.writeSamples(binary_signals)  # Use the NumPy array directly

print("EDF file with binary spike data created successfully.")


EDF file with binary spike data created successfully.
